# Worksheet 14

Name:  Haya AlMajali, Chris Chan
UID: U83334432, U31827126

### Topics

- Naive Bayes
- Model Evaluation

### Naive Bayes

| Attribute A | Attribute B | Attribute C | Class |
|-------------|-------------|-------------|-------|
| Yes         | Single      | High        | No    |
| No          | Married     | Mid         | No    |
| No          | Single      | Low         | No    |
| Yes         | Married     | High        | No    |
| No          | Divorced    | Mid         | Yes   |
| No          | Married     | Low         | No    |
| Yes         | Divorced    | High        | No    |
| No          | Single      | Mid         | Yes   |
| No          | Married     | Low         | No    |
| No          | Single      | Mid         | Yes   |

a) Compute the following probabilities:

- P(Attribute A = Yes | Class = No)
- P(Attribute B = Divorced | Class = Yes)
- P(Attribute C = High | Class = No)
- P(Attribute C = Mid | Class = Yes)

- Probability(Attribute A = Yes | Class = No): 0.4
- Probability(Attribute B = Divorced | Class = Yes): 0.6667
- Probability(Attribute C = High | Class = No): 0.4
- Probability(Attribute C = Mid | Class = Yes): 0.6667

b) Classify the following unseen records:

- (Yes, Married, Mid)
- (No, Divorced, High)
- (No, Single, High)
- (No, Divorced, Low)

- Record 1 (Yes, Married, Mid): No. 

Probability(Class = No | Attribute A = Yes, Attribute B = Married, Attribute C = Mid): 0.0144
Probability(Class = Yes | Attribute A = Yes, Attribute B = Married, Attribute C = Mid): 0

Probability for (Class = No) is higher, so we classify it as a No.

- Record 2 (No, Divorced, High): Yes

Probability(Class = No | Attribute A = No, Attribute B = Divorced, Attribute C = High): 0.0288
Probability(Class = Yes | Attribute A = No, Attribute B = Divorced, Attribute C = High): 0.0427

Probability for (Class = Yes) is higher, so we classify it as a Yes.
    
- Record 3 (No, Single, High): No

Probability(Class = No | Attribute A = No, Attribute B = Single, Attribute C = High): 0.0288
Probability(Class = Yes | Attribute A = No, Attribute B = Single, Attribute C = High): 0.0213

Probability for (Class = No) is higher, so we classify it as a No.
    
- Record 4 (No, Divorced, Low): No

Probability(Class = No | Attribute A = No, Attribute B = Divorced, Attribute C = Low): 0.0475
Probability(Class = Yes | Attribute A = No, Attribute B = Divorced, Attribute C = Low): 0.0213

Probability for (Class = No) is higher, so we classify it as a No.

### Model Evaluation

a) Write a function to generate the confusion matrix for a list of actual classes and a list of predicted classes

In [ ]:
def confusion_matrix(actual, predicted):
    # initialize variables for true positives, true negatives, false positives, and false negatives
    tp, tn, fp, fn = 0, 0, 0, 0
    
    # calculate the confusion matrix metrics
    for act, pred in zip(actual, predicted):
        if act == "Yes" and pred == "Yes":
            tp += 1
        elif act == "No" and pred == "No":
            tn += 1
        elif act == "No" and pred == "Yes":
            fp += 1
        elif act == "Yes" and pred == "No":
            fn += 1
    # construct the confusion matrix as a list of lists
    confusion_matrix = [[tn, fp], [fn, tp]]
    
    return confusion_matrix

actual_class = ["Yes", "No", "No", "Yes", "No", "No", "Yes", "No", "No", "No"]
predicted_class = ["Yes", "No", "Yes", "No", "No", "No", "Yes", "Yes", "Yes", "No"]

print(confusion_matrix(actual_class, predicted_class))

b) Assume you have the following Cost Matrix:

|            | predicted = Y | predicted = N |
|------------|---------------|---------------|
| actual = Y |       -1      |       5       |
| actual = N |        10     |       0       |

What is the cost of the above classification?

Given the confusion matrix: [[4, 3], [1, 2]], and the cost matrix above:

Cost = (tps * Cost of Predicting Yes as Yes) + (fps * Cost of Predicting No as Yes) + (fns * Cost of Predicting Yes as No) + (tns * Cost of Predicting No as No) = (2 * -1) + (3 * 10) + (1 * 5) + (4 * 0) = 33

c) Write a function that takes in the actual values, the predictions, and a cost matrix and outputs a cost. Test it on the above example.

In [ ]:
def calculate_cost(actual, predicted, cost_matrix):
    # initializing the cost to zero
    cost = 0
    
    # iterating through the actual and predicted values to calculate the cost
    for act, pred in zip(actual, predicted):
        if act == "Yes" and pred == "Yes":
            cost += cost_matrix[0][0]  # yes as a yes
        elif act == "No" and pred == "Yes":
            cost += cost_matrix[1][0]  # no as a yes
        elif act == "Yes" and pred == "No":
            cost += cost_matrix[0][1]  # yes as a no
        elif act == "No" and pred == "No":
            cost += cost_matrix[1][1]  # no as a no

    return cost


actual_class = ["Yes", "No", "No", "Yes", "No", "No", "Yes", "No", "No", "No"]
predicted_class = ["Yes", "No", "Yes", "No", "No", "No", "Yes", "Yes", "Yes", "No"]

# the given cost matrix
cost_matrix = [[-1, 5],
               [10, 0]]

total_cost = calculate_cost(actual_class, predicted_class, cost_matrix)

print("Total Cost:", total_cost)

d) Implement functions for the following:

- accuracy
- precision
- recall
- f-measure

and apply them to the above example.

In [ ]:
def accuracy(actual, predicted):
    correct = 0
    total = len(actual)
    for act, pred in zip(actual, predicted):
        if act == pred:
            correct += 1
    return (correct / total)

def precision(actual, predicted, target_class):
    true_positive = sum(act == target_class and pred == target_class for act, pred in zip(actual, predicted))
    false_positive = sum(act != target_class and pred == target_class for act, pred in zip(actual, predicted))
    return true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0

def recall(actual, predicted, target_class):
    true_positive = sum(act == target_class and pred == target_class for act, pred in zip(actual, predicted))
    false_negative = sum(act == target_class and pred != target_class for act, pred in zip(actual, predicted))
    return true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0

def f_measure(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

#given the following
actual_class = ["Yes", "No", "No", "Yes", "No", "No", "Yes", "No", "No", "No"]
predicted_class = ["Yes", "No", "Yes", "No", "No", "No", "Yes", "Yes", "Yes", "No"]

#calculate results
acc = accuracy(actual_class, predicted_class)
prec = precision(actual_class, predicted_class, "Yes")
rec = recall(actual_class, predicted_class, "Yes")
f_measure_val = f_measure(prec, rec)

#print results
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F-measure:", f_measure_val)

## Some useful code for the midterm

In [ ]:
import seaborn as sns
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.datasets import fetch_lfw_people
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

sns.set()

# Get face data
faces = fetch_lfw_people(min_faces_per_person=60)

# plot face data
fig, ax = plt.subplots(3, 5)
for i, axi in enumerate(ax.flat):
    axi.imshow(faces.images[i], cmap='bone')
    axi.set(xticks=[], yticks=[],
            xlabel=faces.target_names[faces.target[i]])
plt.show()

# split train test set
Xtrain, Xtest, ytrain, ytest = train_test_split(faces.data, faces.target, random_state=42)

pca = PCA(n_components=150, whiten=True)
svc = SVC(kernel='rbf', class_weight='balanced')
svcpca = make_pipeline(pca, svc)

# Tune model to find best values of C and gamma using cross validation
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
kfold = 10
grid = GridSearchCV(svcpca, param_grid, cv=kfold)
grid.fit(Xtrain, ytrain)

print(grid.best_params_)

# use the best params explicitly here
pca = PCA(n_components=150, whiten=True)
svc = SVC(kernel='rbf', class_weight='balanced', C=10, gamma=0.005)
svcpca = make_pipeline(pca, svc)

model = BaggingClassifier(svcpca, n_estimators=100).fit(Xtrain, ytrain)
yfit = model.predict(Xtest)

fig, ax = plt.subplots(6, 6)
for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(62, 47), cmap='bone')
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(faces.target_names[yfit[i]].split()[-1],
                   color='black' if yfit[i] == ytest[i] else 'red')
fig.suptitle('Predicted Names; Incorrect Labels in Red', size=14)
plt.show()

mat = confusion_matrix(ytest, yfit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=faces.target_names,
            yticklabels=faces.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label')
plt.show()

print("Accuracy = ", accuracy_score(ytest, yfit))